In [1]:
import numpy as np
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
from torch import optim
import torch
from torch import nn
from torch.autograd import Variable
import pandas
from sklearn.preprocessing import MinMaxScaler
import torchvision.models as models
import torchvision.transforms as transforms
import torch
from torch.autograd import Variable
import numpy as np
from PIL import Image
import copy

from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
from torch.utils.data import Dataset, DataLoader

from IPython.display import display

/home/p.zaydel/conda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dataDir='/home/p.zaydel/ProjectNeuralNets/coco_dataset/'
imagesDirTrain = '{}train2017/train2017'.format(dataDir)
imagesDirVal = '{}val2017/val2017'.format(dataDir)

annTrainFile = '{}/annotations_trainval2017/annotations/captions_train2017.json'.format(dataDir)
annValFile = '{}/annotations_trainval2017/annotations/captions_val2017.json'.format(dataDir)

In [19]:
def get_image_shapes(coco):
    shapes = []
    imids = coco.getImgIds()
    for imid in imids:
        img_data = coco.loadImgs([imid])[0]
        h = img_data['height']
        w = img_data['width']
        shapes.append([w,h])
    return shapes

In [28]:
train_shapes = get_image_shapes(COCO(annTrainFile))
np.array(train_shapes).mean(axis = 0)

loading annotations into memory...
Done (t=1.25s)
creating index...
index created!


array([577.71206472, 484.09889506])

In [30]:
test_shapes = get_image_shapes(COCO(annValFile))
np.array(test_shapes).mean(axis = 0)

loading annotations into memory...
Done (t=0.06s)
creating index...
index created!


array([573.7548, 483.543 ])

In [ ]:
transforms.Resize()

In [ ]:
transform = transforms.Compose([  transforms.Resize(),transforms.ToTensor()
                                           ])

In [8]:
class MSCOCODataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, annFile, imagesDir, transform=None):
        self.coco = COCO(annFile)
        self.imagesDir = imagesDir
        self.imageids = self.coco.getImgIds()
        self.transform = transform

    def __len__(self):
        return len(self.coco.dataset['images'])

    def __getitem__(self, idx):
        imid = self.imageids[idx]
        
        img_data = self.coco.loadImgs([imid])[0]
        
        annIds = self.coco.getAnnIds(imgIds=imid)
        
        anns_data = self.coco.loadAnns(annIds)
        anns = [ann['caption'] for ann in anns_data]
        
        
        img_file_name = '{}/{}'.format(self.imagesDir, img_data['file_name'])
        image = io.imread(img_file_name)
        
        if self.transform:
            image = self.transform(image)
        
        sample = {'image': image, 'anns': anns}

        return sample

In [9]:
trainDataset = MSCOCODataset(annTrainFile,imagesDirTrain )
testDataset = MSCOCODataset(annValFile,imagesDirVal)

loading annotations into memory...
Done (t=1.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!


In [ ]:
shapes

In [ ]:
class CaptionNet(nn.Module):
    def __init__(self, image_size, hidden_size):
        super(CaptionNet, self).__init__()
        self.image_size = image_size
        self.hidden_size = hidden_size
        
        self.cnn = models.inception_v3
        self.lstm = nn.LSTM()
          
    def forward(self, X, hidden=None, C=None):
        output = Variable(torch.zeros(X.size(0), self.output_size))
        if hidden is None:
            hidden = Variable(torch.zeros(self.hidden_size), requires_grad=False)
        if C is None:
            C = Variable(torch.zeros(self.hidden_size), requires_grad=False)
        
        for i in range(X.size(0)):    
            f_i = self.forget(torch.mv(self.w_f_h, hidden) + torch.mv(self.w_f_i, X[i]) + self.b_f_h + self.b_f_i)
            input_i = self.forget(torch.mv(self.w_i_h, hidden) + torch.mv(self.w_i_i, X[i]) + self.b_i_i + self.b_i_h)
            C = f_i * C + input_i * self.tanh(torch.mv(self.w_c_h, hidden) + torch.mv(self.w_c_i, X[i]) + self.b_c_i + self.b_c_h)
            
            output[i] = self.forget(torch.mv(self.w_o_h, hidden) + torch.mv(self.w_o_i, X[i]) + self.b_o_i + self.b_o_h)
            hidden = output[i].clone() * self.tanh(C)
            
        return output, hidden, C